# SEC CIK files to graph

(C) 2023-2024 by [Damir Cavar](http://damir.cavar.me/)

This code generates a graph or triple representation from the Central Index Key (CIK) JSON data files for corporations that describe the coporate filings on [SEC](https://www.sec.gov/).

**Prerequisites:**

In [ ]:
!pip install -U rdflib

We need to import various modules for RDF, JSON, and regular expression processing:

In [1]:
from rdflib.namespace import RDF, RDFS, SKOS, OWL, DC, DCTERMS, XSD, TIME, NamespaceManager
from rdflib import Graph, URIRef, Literal, Namespace
from datetime import datetime
import os
import glob
import json
import re

The data and files are located in a specific subfolder, and all files end in `json`.

In [2]:
data_folder = os.path.join(".", "data", "CIKs")
data_files = tuple(glob.glob(os.path.join(data_folder, '*.json')))

The following regular expression is used to clean corporation names and to generate the URI of the instance:

In [3]:
re_name_clean = re.compile(r"[ &,.-]+")

We need a couple of URIs to define the graph relations:

In [4]:
default_prefix = r"http://www.iu.edu/kg/sec/"
has_cik_uri = URIRef(default_prefix + r"hasCIK")
entityType_uri = URIRef(default_prefix + r"entityType")
has_sic_uri = URIRef(default_prefix + r"hasSIC")
sicDescription_uri = URIRef(default_prefix + r"hasSICDescription")
exchange_uri = URIRef(default_prefix + r"exchange")

We create a graph instance for all the class specifications and corporation instances:

In [5]:
g = Graph()

A couple of namespace declarations are necessary, too:

In [6]:
vaem_ns = Namespace(r"http://www.linkedmodel.org/schema/vaem#")
iu_ns = Namespace(r"http://www.iu.edu/kg/sec/")
namespace_manager = NamespaceManager(g)
namespace_manager.bind("vaem", vaem_ns, override=True)
namespace_manager.bind("iusec", iu_ns, override=True)
g.namespace_manager = namespace_manager

We add the specification and meta-information to the graph:

In [7]:
ontology_uri = URIRef(default_prefix + r"SEC")
g.add( (ontology_uri, RDF.type, OWL.Ontology) )
g.add( (ontology_uri, DC.creator, Literal("Damir Cavar")) )
g.add( (ontology_uri, OWL.versionInfo, Literal("Created using automatic conversion scripts by Damir Cavar")) )
g.add( (ontology_uri, vaem_ns.hasOwner,   Literal("Indiana University, NLP-Lab, Course L715")) )
g.add( (ontology_uri, vaem_ns.hasSteward, Literal("Damir Cavar")) )
dt = datetime.now()
g.add( (ontology_uri, vaem_ns.publishedDate, Literal(dt.strftime("%Y-%m-%dT%H:%M:%SZ"))) )
g.add( (ontology_uri, vaem_ns.releaseDate,   Literal(dt.strftime("%Y-%m-%dT%H:%M:%SZ"))) )
g.add( (ontology_uri, OWL.imports, URIRef(r"http://purl.org/dc/elements/1.1/")) )
g.add( (ontology_uri, OWL.imports, URIRef(r"http://purl.org/dc/terms/")) )
corporation_uri = URIRef(default_prefix + r"Corporation")
g.add( (corporation_uri, RDF.type,        OWL.Class) )
g.add( (corporation_uri, RDFS.subClassOf, OWL.Thing) )

<Graph identifier=N0af729135dcd425b8aac7c1bebc79b24 (<class 'rdflib.graph.Graph'>)>

The following function converts CIK data from the corporate SEC JSON data to triples and stores them in the graph:

In [8]:
def cik_to_graph(filename, g):
    with open(filename, mode='r', encoding='utf-8') as f:
        data = json.load(f)
    if not "name" in data: return
    name = re_name_clean.sub("_", data["name"].strip())
    name = re.sub(r"__+", "_", name.lower())
    while name[-1] == "_": name = name[:-1]
    corp_id_uri = URIRef(default_prefix + name)
    g.add( (corp_id_uri, RDFS.label, Literal(data["name"].strip())) )
    g.add( (corp_id_uri, RDFS.subClassOf, corporation_uri) )
    for k in data:
        if k == "cik":
            g.add( (corp_id_uri, has_cik_uri, Literal(data["cik"])) )
        elif k == "sic":
            g.add( (corp_id_uri, has_sic_uri, Literal(data["sic"])) )
        elif k == "sicDescription":
            g.add( (corp_id_uri, sicDescription_uri, Literal(data["sicDescription"])) )
        elif k == "insiderTransactionForOwnerExists":
            g.add( (corp_id_uri, URIRef(default_prefix + "insiderTransactionForOwnerExists"), Literal(data["insiderTransactionForOwnerExists"])) )
        elif k == "insiderTransactionForIssuerExists":
            g.add( (corp_id_uri, URIRef(default_prefix + "insiderTransactionForIssuerExists"), Literal(data["insiderTransactionForIssuerExists"])) )
        elif k == "exchanges":
            for e in data["exchanges"]:
                g.add( (corp_id_uri, exchange_uri, Literal(e)) )
        elif k == "ein":
            g.add( (corp_id_uri, URIRef(default_prefix + "hasEIN"), Literal(data["ein"])) )
        elif k == "description":
            g.add( (corp_id_uri, URIRef(default_prefix + "description"), Literal(data["description"])) )
        elif k == "website":
            g.add( (corp_id_uri, URIRef(default_prefix + "website"), Literal(data["website"])) )
        elif k == "investorWebsite":
            g.add( (corp_id_uri, URIRef(default_prefix + "investorWebsite"), Literal(data["investorWebsite"])) )
        elif k == "category":
            g.add( (corp_id_uri, URIRef(default_prefix + "category"), Literal(data["category"])) )
        elif k == "fiscalYearEnd":
            g.add( (corp_id_uri, URIRef(default_prefix + "fiscalYearEnd"), Literal(data["fiscalYearEnd"])) )
        elif k == "stateOfIncorporation":
            g.add( (corp_id_uri, URIRef(default_prefix + "stateOfIncorporation"), Literal(data["stateOfIncorporation"])) )
        elif k == "stateOfIncorporationDescription":
            g.add( (corp_id_uri, URIRef(default_prefix + "stateOfIncorporationDescription"), Literal(data["stateOfIncorporationDescription"])) )
        #     "addresses": {
        # "mailing": {
        #     "street1": "ROUTE 206 AND PROVINCE LINE ROAD",
        #     "street2": null,
        #     "city": "PRINCETON",
        #     "stateOrCountry": "NJ",
        #     "zipCode": "08543",
        #     "stateOrCountryDescription": "NJ"
        # },
        # "business": {
        #     "street1": "ROUTE 206 AND PROVINCE LINE ROAD",
        #     "street2": null,
        #     "city": "PRINCETON",
        #     "stateOrCountry": "NJ",
        #     "zipCode": "08543",
        #     "stateOrCountryDescription": "NJ"
        # }
        # },
        elif k == "phone":
            g.add( (corp_id_uri, URIRef(default_prefix + "phone"), Literal(data["phone"])) )
        elif k == "flags":
            g.add( (corp_id_uri, URIRef(default_prefix + "flags"), Literal(data["flags"])) )
        elif k == "entityType":
            g.add( (corp_id_uri, entityType_uri, Literal(data["entityType"])) )


We generate the corporate SEC CIK triple data for all corporations that we have JSON data for:

In [9]:
for f in data_files:
    cik_to_graph(f, g)

We serialize the graph as a Turtle graph in the same location where the JSON data files are stored:

In [11]:
g.serialize(destination=os.path.join(".", "data", "CIKs", "cik_graph.ttl"), format="ttl")

<Graph identifier=N0af729135dcd425b8aac7c1bebc79b24 (<class 'rdflib.graph.Graph'>)>

We can also serialize the graph in JSON-LD format:

In [12]:
g.serialize(destination=os.path.join(".", "data", "CIKs", "cik_graph.jsonld"), format="json-ld")

<Graph identifier=N0af729135dcd425b8aac7c1bebc79b24 (<class 'rdflib.graph.Graph'>)>

(C) 2023-2024 by [Damir Cavar](http://damir.cavar.me/)